In [2]:
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
import cvxpy
import numpy as np

In [3]:
### Считаем временной ряд цен акций
df = pd.read_csv("task-1-stocks.csv", parse_dates=True)
df

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S90,S91,S92,S93,S94,S95,S96,S97,S98,S99
0,10.829591,21.462513,2.1900,0.0007,2.84,79.650002,48.664711,34.250000,108.224541,0.07683,...,35.585823,125.713310,69.806602,51.589684,28.519390,101.637383,56.108028,289.124451,225.103073,178.199997
1,10.648883,20.986433,2.4000,0.0006,2.79,76.279999,48.664711,33.299999,106.566994,0.07683,...,33.694645,119.911293,67.153770,52.173275,28.277946,103.727180,54.567417,280.817444,224.172348,174.880005
2,10.584344,21.083593,2.0200,0.0009,2.73,75.089996,48.703156,33.049999,105.432350,0.07683,...,33.971889,120.450333,67.037582,52.299721,27.737110,104.549232,55.143902,282.595428,225.053558,173.550003
3,10.519805,20.869841,2.0000,0.0007,2.63,73.550003,48.712761,32.400002,105.037697,0.07683,...,34.645187,119.774086,68.121956,52.844410,27.244562,104.707703,55.382450,285.390747,225.449615,173.130005
4,10.584344,20.967001,2.0000,0.0011,2.88,76.010002,48.722374,32.599998,105.471817,0.06500,...,34.744202,119.901497,68.828735,52.533154,27.746767,104.162964,56.078205,288.670074,223.964417,176.399994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,21.491381,19.354856,0.6810,0.0005,1.41,73.269997,49.987724,29.650000,118.816467,0.13480,...,36.856209,124.944298,62.187641,63.459393,30.375267,98.570091,65.037018,354.824951,272.469757,183.509995
96,21.039610,19.149059,0.6580,0.0005,1.41,73.820000,49.997524,29.700001,117.425995,0.13480,...,36.637592,124.944298,61.105259,62.558067,30.355497,97.953278,65.784798,361.278687,262.396698,182.699997
97,20.910534,19.286257,0.6517,0.0005,1.42,72.519997,49.997524,30.450001,118.002045,0.13480,...,36.269928,126.972519,61.351257,64.123009,30.434576,95.515892,66.024086,357.613617,263.251007,183.740005
98,20.975073,18.884460,0.6848,0.0005,1.41,74.410004,50.026928,32.349998,117.515381,0.13480,...,35.624023,125.948463,60.741188,63.211777,30.404922,95.824295,65.156654,354.496307,265.674896,184.000000


In [15]:
### Найдем ожидаемый доход по каждой акции и матрицу ковариаций
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

In [5]:
mu

S0     4.422185
S1    -0.279911
S2    -0.958058
S3    -0.575343
S4    -0.873646
         ...   
S95   -0.145347
S96    0.406821
S97    0.617878
S98    0.462255
S99    0.031872
Length: 100, dtype: float64

In [6]:
S

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S90,S91,S92,S93,S94,S95,S96,S97,S98,S99
S0,2.294337,0.031354,0.006247,-0.022036,-0.040267,0.025843,0.000803,0.045098,-0.037651,0.128031,...,0.016859,0.044909,0.074527,-0.031630,-0.040703,-0.010662,0.010438,0.078516,-0.025318,-0.064066
S1,0.031354,0.102675,0.043986,0.004363,0.093247,0.031813,0.000041,0.029221,0.016473,-0.079474,...,0.039619,0.041691,0.062842,0.028618,0.017010,-0.005819,0.027507,0.029252,0.006560,0.001784
S2,0.006247,0.043986,1.445235,-0.220267,0.164069,0.062921,0.000215,0.071905,-0.001639,-0.043707,...,0.047114,0.049988,0.079161,0.015407,0.037031,-0.029616,-0.020929,-0.010039,0.043866,0.011895
S3,-0.022036,0.004363,-0.220267,6.420305,-0.015584,0.081927,0.001588,0.093830,0.038265,-0.266835,...,-0.023426,-0.008554,0.013712,-0.046089,-0.006168,0.001825,0.048804,0.050256,-0.039129,-0.036574
S4,-0.040267,0.093247,0.164069,-0.015584,0.520194,0.083658,-0.000442,0.044293,0.010800,0.017433,...,0.030107,0.068018,0.095862,0.036902,0.028701,-0.056706,0.043129,0.023117,0.016063,0.008904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S95,-0.010662,-0.005819,-0.029616,0.001825,-0.056706,0.001570,0.000162,-0.014090,0.001405,-0.025662,...,0.001492,-0.007269,-0.007536,-0.006413,-0.001149,0.048726,-0.004663,-0.004153,0.005699,-0.001123
S96,0.010438,0.027507,-0.020929,0.048804,0.043129,0.040047,0.000391,0.032674,0.004442,0.001785,...,0.021072,0.036355,0.039025,0.019144,-0.004925,-0.004663,0.085333,0.016407,0.005167,-0.000641
S97,0.078516,0.029252,-0.010039,0.050256,0.023117,0.028798,0.000142,0.027789,0.006212,0.006003,...,0.033769,0.034082,0.039715,0.017746,-0.000698,-0.004153,0.016407,0.063707,-0.001282,-0.009262
S98,-0.025318,0.006560,0.043866,-0.039129,0.016063,0.000626,0.000136,-0.002357,0.005853,-0.017157,...,0.006272,-0.002905,0.004170,0.004067,0.000044,0.005699,0.005167,-0.001282,0.030235,0.007858


In [ ]:
### Максимизируем доходность портфела с ограничением на риск (20%)
S += 0.0000004 * np.eye(*S.shape)
ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.02)
weights = ef.efficient_risk(target_volatility=0.2, market_neutral=False)
ef.portfolio_performance(verbose=True)

Expected annual return: 174.2%
Annual volatility: 20.0%
Sharpe Ratio: 8.61


(1.7422234727491968, 0.19999999999931445, 8.6111173637755)

In [28]:
weights

OrderedDict([('S0', 0.060145900883346),
             ('S1', 9.644e-13),
             ('S2', 6.052e-13),
             ('S3', 8.49e-13),
             ('S4', 6.465e-13),
             ('S5', 8.799e-13),
             ('S6', 1.4222e-12),
             ('S7', 9.957e-13),
             ('S8', 1.7329e-12),
             ('S9', 0.0341714788335902),
             ('S10', 1.3135e-12),
             ('S11', 1.0845e-12),
             ('S12', 5.127e-13),
             ('S13', 0.0280147936665545),
             ('S14', 0.0008782018629857),
             ('S15', 0.0015690060945923),
             ('S16', 1.0726e-12),
             ('S17', 1.3135e-12),
             ('S18', 7.194e-13),
             ('S19', 1.3978e-12),
             ('S20', 2.1422e-12),
             ('S21', 1.2991e-12),
             ('S22', 1.7535e-12),
             ('S23', 1.1676e-12),
             ('S24', 1.3604e-12),
             ('S25', 1.2835e-12),
             ('S26', 1.1599e-12),
             ('S27', 0.2696247410467058),
             ('S28',

In [39]:
### Перегоним полученные веса в массив
ans = []
for column in df.columns:
    ans.append(weights[column])
    
ans = np.array(ans)
ans

array([6.01459009e-02, 9.64400000e-13, 6.05200000e-13, 8.49000000e-13,
       6.46500000e-13, 8.79900000e-13, 1.42220000e-12, 9.95700000e-13,
       1.73290000e-12, 3.41714788e-02, 1.31350000e-12, 1.08450000e-12,
       5.12700000e-13, 2.80147937e-02, 8.78201863e-04, 1.56900609e-03,
       1.07260000e-12, 1.31350000e-12, 7.19400000e-13, 1.39780000e-12,
       2.14220000e-12, 1.29910000e-12, 1.75350000e-12, 1.16760000e-12,
       1.36040000e-12, 1.28350000e-12, 1.15990000e-12, 2.69624741e-01,
       8.94400000e-13, 7.93900000e-13, 2.19790000e-12, 9.87800000e-13,
       1.75750000e-12, 8.90600000e-13, 1.32700000e-12, 1.97640000e-12,
       1.24580000e-12, 1.36850000e-12, 1.78630000e-12, 1.21510000e-12,
       1.64110000e-12, 8.18000000e-13, 7.46100000e-13, 1.04670000e-12,
       2.26040000e-12, 1.14520000e-12, 3.09596080e-03, 1.55260000e-12,
       1.21430000e-12, 1.38810000e-12, 1.34090000e-12, 1.79100000e-12,
       9.39200000e-13, 1.79720000e-12, 1.93120000e-12, 9.24300000e-13,
      

In [40]:
### Считаем сколько денег мы должны вложить в каждую акцию
ans = ans * 1000000
ans

array([6.01459009e+04, 9.64400000e-07, 6.05200000e-07, 8.49000000e-07,
       6.46500000e-07, 8.79900000e-07, 1.42220000e-06, 9.95700000e-07,
       1.73290000e-06, 3.41714788e+04, 1.31350000e-06, 1.08450000e-06,
       5.12700000e-07, 2.80147937e+04, 8.78201863e+02, 1.56900609e+03,
       1.07260000e-06, 1.31350000e-06, 7.19400000e-07, 1.39780000e-06,
       2.14220000e-06, 1.29910000e-06, 1.75350000e-06, 1.16760000e-06,
       1.36040000e-06, 1.28350000e-06, 1.15990000e-06, 2.69624741e+05,
       8.94400000e-07, 7.93900000e-07, 2.19790000e-06, 9.87800000e-07,
       1.75750000e-06, 8.90600000e-07, 1.32700000e-06, 1.97640000e-06,
       1.24580000e-06, 1.36850000e-06, 1.78630000e-06, 1.21510000e-06,
       1.64110000e-06, 8.18000000e-07, 7.46100000e-07, 1.04670000e-06,
       2.26040000e-06, 1.14520000e-06, 3.09596080e+03, 1.55260000e-06,
       1.21430000e-06, 1.38810000e-06, 1.34090000e-06, 1.79100000e-06,
       9.39200000e-07, 1.79720000e-06, 1.93120000e-06, 9.24300000e-07,
      

In [41]:
### Посчитаем теперь сколько каждой акции мы должны купить (делим бюджет для i-ой акции на ее стоимость в первый день)
ans = (ans // df.iloc[0].to_numpy()).astype(int)
ans

array([   5553,       0,       0,       0,       0,       0,       0,
             0,       0,  444767,       0,       0,       0, 3638284,
          7318,   71318,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,    3568,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,      20,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,   12489,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,     

In [42]:
### Введим индексы акций, попавших в наш портфель
np.nonzero(ans)

(array([ 0,  9, 13, 14, 15, 27, 46, 59], dtype=int64),)

In [ ]:
### Введим названия акций, попавших в наш портфель
df.columns[np.nonzero(ans)]

Index(['S0', 'S9', 'S13', 'S14', 'S15', 'S27', 'S46', 'S59'], dtype='object')

In [46]:
### Портфель
portfolio = dict(zip(df.columns, ans))
portfolio

{'S0': 5553,
 'S1': 0,
 'S2': 0,
 'S3': 0,
 'S4': 0,
 'S5': 0,
 'S6': 0,
 'S7': 0,
 'S8': 0,
 'S9': 444767,
 'S10': 0,
 'S11': 0,
 'S12': 0,
 'S13': 3638284,
 'S14': 7318,
 'S15': 71318,
 'S16': 0,
 'S17': 0,
 'S18': 0,
 'S19': 0,
 'S20': 0,
 'S21': 0,
 'S22': 0,
 'S23': 0,
 'S24': 0,
 'S25': 0,
 'S26': 0,
 'S27': 3568,
 'S28': 0,
 'S29': 0,
 'S30': 0,
 'S31': 0,
 'S32': 0,
 'S33': 0,
 'S34': 0,
 'S35': 0,
 'S36': 0,
 'S37': 0,
 'S38': 0,
 'S39': 0,
 'S40': 0,
 'S41': 0,
 'S42': 0,
 'S43': 0,
 'S44': 0,
 'S45': 0,
 'S46': 20,
 'S47': 0,
 'S48': 0,
 'S49': 0,
 'S50': 0,
 'S51': 0,
 'S52': 0,
 'S53': 0,
 'S54': 0,
 'S55': 0,
 'S56': 0,
 'S57': 0,
 'S58': 0,
 'S59': 12489,
 'S60': 0,
 'S61': 0,
 'S62': 0,
 'S63': 0,
 'S64': 0,
 'S65': 0,
 'S66': 0,
 'S67': 0,
 'S68': 0,
 'S69': 0,
 'S70': 0,
 'S71': 0,
 'S72': 0,
 'S73': 0,
 'S74': 0,
 'S75': 0,
 'S76': 0,
 'S77': 0,
 'S78': 0,
 'S79': 0,
 'S80': 0,
 'S81': 0,
 'S82': 0,
 'S83': 0,
 'S84': 0,
 'S85': 0,
 'S86': 0,
 'S87': 0,
 'S88': 0,
 '

In [47]:
### Посчитаем стоимсоть портфеля на каждый день
costs = (df.values*ans)[:, [ 0,  9, 13, 14, 15, 27, 46, 59]]
costs

array([[6.01367177e+04, 3.41714485e+04, 2.80147866e+04, 8.78159980e+02,
        1.56899599e+03, 2.69562411e+05, 3.07068268e+03, 6.02478147e+05],
       [5.91332466e+04, 3.41714485e+04, 2.80147866e+04, 1.09770004e+03,
        1.56899599e+03, 2.74521929e+05, 3.01259857e+03, 5.91589624e+05],
       [5.87748617e+04, 3.41714485e+04, 2.80147866e+04, 8.42667679e+02,
        1.56899599e+03, 2.76412964e+05, 2.97003082e+03, 5.88789671e+05],
       [5.84164769e+04, 3.41714485e+04, 2.80147866e+04, 7.33995394e+02,
        1.49767798e+03, 2.74058071e+05, 2.96386414e+03, 5.87731976e+05],
       [5.87748617e+04, 2.89098539e+04, 2.80147866e+04, 1.09879771e+03,
        1.78295003e+03, 2.75164170e+05, 3.01478668e+03, 6.00922644e+05],
       [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
        1.71163201e+03, 2.71703205e+05, 3.00424438e+03, 5.96380527e+05],
       [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
        1.74729095e+03, 2.74593277e+05, 2.97997620e+03, 5.

In [48]:
costs.shape

(100, 8)

In [ ]:
### Все дни кроме последнего
costs_i = costs[:-1]
costs_i.shape, costs_i

((99, 8),
 array([[6.01367177e+04, 3.41714485e+04, 2.80147866e+04, 8.78159980e+02,
         1.56899599e+03, 2.69562411e+05, 3.07068268e+03, 6.02478147e+05],
        [5.91332466e+04, 3.41714485e+04, 2.80147866e+04, 1.09770004e+03,
         1.56899599e+03, 2.74521929e+05, 3.01259857e+03, 5.91589624e+05],
        [5.87748617e+04, 3.41714485e+04, 2.80147866e+04, 8.42667679e+02,
         1.56899599e+03, 2.76412964e+05, 2.97003082e+03, 5.88789671e+05],
        [5.84164769e+04, 3.41714485e+04, 2.80147866e+04, 7.33995394e+02,
         1.49767798e+03, 2.74058071e+05, 2.96386414e+03, 5.87731976e+05],
        [5.87748617e+04, 2.89098539e+04, 2.80147866e+04, 1.09879771e+03,
         1.78295003e+03, 2.75164170e+05, 3.01478668e+03, 6.00922644e+05],
        [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
         1.71163201e+03, 2.71703205e+05, 3.00424438e+03, 5.96380527e+05],
        [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
         1.74729095e+03, 2.74593277

In [50]:
### Все дни кроме первого
costs_i1 = costs[1:]
costs_i1.shape, costs_i1

((99, 8),
 array([[5.91332466e+04, 3.41714485e+04, 2.80147866e+04, 1.09770004e+03,
         1.56899599e+03, 2.74521929e+05, 3.01259857e+03, 5.91589624e+05],
        [5.87748617e+04, 3.41714485e+04, 2.80147866e+04, 8.42667679e+02,
         1.56899599e+03, 2.76412964e+05, 2.97003082e+03, 5.88789671e+05],
        [5.84164769e+04, 3.41714485e+04, 2.80147866e+04, 7.33995394e+02,
         1.49767798e+03, 2.74058071e+05, 2.96386414e+03, 5.87731976e+05],
        [5.87748617e+04, 2.89098539e+04, 2.80147866e+04, 1.09879771e+03,
         1.78295003e+03, 2.75164170e+05, 3.01478668e+03, 6.00922644e+05],
        [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
         1.71163201e+03, 2.71703205e+05, 3.00424438e+03, 5.96380527e+05],
        [6.02083957e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
         1.74729095e+03, 2.74593277e+05, 2.97997620e+03, 5.98620499e+05],
        [5.84164769e+04, 2.89098539e+04, 2.80147866e+04, 9.16433136e+02,
         1.68310486e+03, 2.76948170

In [52]:
### Доходность за первые 99 дней 
r = np.sum((costs_i1 - costs_i), axis=1)/np.sum(costs_i, axis=1)
r

array([-6.77182376e-03, -1.57575929e-03, -3.99087057e-03,  1.02214187e-02,
       -6.84964410e-03,  5.18891778e-03,  1.85537072e-03,  3.31550013e-02,
       -1.90193741e-02, -3.04984656e-03,  9.40527563e-03,  1.24433109e-02,
        7.61146617e-04, -1.69763738e-03,  9.16031664e-03,  8.43030176e-04,
        2.42469084e-03,  5.61363422e-03,  8.61756532e-03, -4.59681465e-03,
        6.59261052e-03,  2.90124960e-02,  3.49633473e-02, -2.49706029e-03,
        1.23127851e-02, -5.16105457e-04,  1.14466350e-03, -3.47833384e-02,
       -1.05547744e-02,  9.27900107e-03,  1.72953094e-02, -3.84475963e-03,
       -7.96225721e-03,  4.80215913e-05,  1.01228664e-02,  9.99190393e-03,
        4.44916027e-03, -1.41944122e-04,  4.93206871e-03,  2.06410234e-03,
        7.06656973e-03,  5.91279919e-03, -1.03176581e-02,  8.39253661e-03,
        3.70952643e-05, -5.13475103e-03, -1.37333810e-03,  5.99776584e-03,
        9.38833074e-04, -5.72491558e-03,  1.85653009e-03,  2.18182919e-03,
        8.72458584e-03,  

In [53]:
### Средняя доходность
r.mean()

0.003933675350094187

In [56]:
### Вектор средней доходности
rm = np.ones(len(r))*r.mean()
rm

array([0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393368,
       0.00393368, 0.00393368, 0.00393368, 0.00393368, 0.00393

In [ ]:
### Уровень риска собранного портфеля
sigma = np.sqrt(len(r)*np.sum((r-rm)**2)/(len(r)-1))
sigma

0.1249985811913721